In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Introduction:
This project had launched by <font color=blue> ASHRAE Organization </font> in order to :  
develop accurate models of metered building energy usage in the following areas: chilled water, electric, hot water, and steam meters. The data that we will use, comes from over 1,000 buildings over a three-year timeframe.

# Importing datas :

In [ ]:
# importing datas of  training building energy consumption 
dt_tr=pd.read_csv('/kaggle/input/ashrae-energy-prediction/train.csv')

In [ ]:
#Importing datas of building caracteristic. 
dt_bd=pd.read_csv('/kaggle/input/ashrae-energy-prediction/building_metadata.csv')

In [ ]:
#Importing training datas of weather conditions .
dt_tw=pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_train.csv')

In [ ]:
# Importing datas of test building that we should their energy consumption.
dt_ts=pd.read_csv('/kaggle/input/ashrae-energy-prediction/test.csv')

In [ ]:
# Importing test datas of weather conditions .
dt_tsw=pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_test.csv')

# Missing Values :

In [ ]:
# Missing values for training building consumptions:
building_miss=pd.DataFrame({c:[sum(dt_tr[c].isna()),(sum(dt_tr[c].isna())/len(dt_tr[c]))*100] \
                            for c in dt_tr.columns} ,index=['Total','%'])
building_miss

In [ ]:
# Missing values for test building consumptions:
building_test_miss=pd.DataFrame({c:[sum(dt_ts[c].isna()),(sum(dt_ts[c].isna())/len(dt_ts[c]))*100] \
                            for c in dt_ts.columns} ,index=['Total','%'])
building_test_miss

In [ ]:
# Missing values building caracteristic:
build_miss=pd.DataFrame({c:[sum(dt_bd[c].isna()),(sum(dt_bd[c].isna())/len(dt_bd[c]))*100] \
                            for c in dt_bd.columns} ,index=['Total','%'])
build_miss

The previous table, which summarize the missing values of building caracteristic , show that this dataframe has two
variables , that have more than 50% of missing values . So, we decide to delete theses variables , which contains
very big quanity of missing values.

In [ ]:
del (dt_bd['year_built'])
del (dt_bd['floor_count'])

In [ ]:
# Let's now examinate the missing values of the weather conditions .
weath_tr_miss=pd.DataFrame({c:[sum(dt_tw[c].isna()),(sum(dt_tw[c].isna())/len(dt_tw))*100] \
                            for c in dt_tw.columns},index=['total','%'])
weath_tr_miss

In [ ]:
# test weather conditions.
weath_ts_miss=pd.DataFrame({c:[sum(dt_tsw[c].isna()),(sum(dt_tsw[c].isna())/len(dt_tsw))*100] \
                            for c in dt_tsw.columns},index=['total','%'])
weath_ts_miss

We notice from the above table , which summarize the weather variables missing values, that the two variables cloud_coverage and precip_depth_1_hr have more than 30% of missing values . So we decide to drop
thseses variables because of an extrapolation of theses big quantity of missing values could inevatilly affect our predictions .

In [ ]:
del (dt_tw['cloud_coverage'])
del(dt_tw['precip_depth_1_hr'])

In [ ]:
del(dt_tsw['cloud_coverage'])
del(dt_tsw['precip_depth_1_hr'])

In [ ]:
# Linear regression , which link air_temperature to dew_temperature for both training and tests datas
from sklearn.linear_model import LinearRegression
model_tr=LinearRegression()
model_ts=LinearRegression()
indtr=dt_tw[(dt_tw['air_temperature'].notna()==True) & (dt_tw['dew_temperature'].notna()==True)].index
indts=dt_tsw[(dt_tsw['air_temperature'].notna()==True) & (dt_tsw['dew_temperature'].notna()==True)].index
model_tr.fit(pd.DataFrame({'cte':np.ones(len(indtr)),'air':dt_tw.iloc[indtr]['air_temperature']}),\
             dt_tw.iloc[indtr]['dew_temperature'])
model_ts.fit(pd.DataFrame({'cte':np.ones(len(indts)),'air':dt_tsw.iloc[indts]['air_temperature']}),\
             dt_tsw.iloc[indts]['dew_temperature'])

In [ ]:
intertr=model_tr.intercept_   # the intercept of the linear regression for the training datas .
coeftr=model_tr.coef_         # the coefficient of the linear regression for the training datas .
interts=model_ts.intercept_   # the intercept of the linear regression for the test datas.
coefts=model_ts.coef_         # the coefficient of the linear regression for the test datas .

In [ ]:
import matplotlib.pyplot as plt 
# Let's visualize theses linear regreesion in compare with the true relationship between the above two variables.
fig=plt.figure(figsize=(20,20))
fig.add_subplot(1,2,1)
plt.plot(dt_tw.iloc[indtr]['air_temperature'],dt_tw.iloc[indtr]['dew_temperature'],\
         label='true train dew_temperature')
plt.plot(dt_tw.iloc[indtr]['air_temperature'],intertr + coeftr[1] * dt_tw.iloc[indtr]['air_temperature'],\
        label='regressed train dew_temperature',color='black',linewidth=2,linestyle='dashed')
plt.legend()
plt.xlabel('air_temperature')
plt.ylabel('dew_temperature')
plt.title('Regressed dew_temperature VS true dew_temperature')
fig.add_subplot(1,2,2)
plt.plot(dt_tsw.iloc[indts]['air_temperature'],dt_tsw.iloc[indts]['dew_temperature'],\
         label='true test dew_temperature',color='yellow')
plt.plot(dt_tsw.iloc[indts]['air_temperature'],interts + coefts[1] * dt_tsw.iloc[indts]['air_temperature'],\
        label='regressed test dew_temperature',color='green',linewidth=2,linestyle='dashed')
plt.legend()
plt.xlabel('air_temperature')
plt.ylabel('dew_temperature')
plt.title('Regressed dew_temperature VS true dew_temperature')
plt.show()

In [ ]:
# Compute the number of dew_temperature and air_temperature that could be imputed by linear relation
# between theses two variables .
l=len(dt_tw[((dt_tw['dew_temperature'].isna()==False) & (dt_tw['air_temperature'].isna()==True))|\
     ((dt_tw['dew_temperature'].isna()==True) & (dt_tw['air_temperature'].isna()==False))])
print('There is {} missing values in dt_tw , which could imputed by the linear relation between dew_temperature and air_temperature'.format(l))

In [ ]:
# Compute the number of dew_temperature and air_temperature that could be imputed by linear relation
# between theses two variables in dt_tsw.
l=len(dt_tsw[((dt_tsw['dew_temperature'].isna()==False) & (dt_tsw['air_temperature'].isna()==True))|\
     ((dt_tsw['dew_temperature'].isna()==True) & (dt_tsw['air_temperature'].isna()==False))])
print('There is {} missing values in dt_tsw , which could imputed by the linear \
relation between dew_temperature and air_temperature'.format(l))

Let's now use the linear regression to impute air_temperature and dew_temperature missing values.

In [ ]:
# Impute train missing values .
mis_tw=dt_tw[((dt_tw['dew_temperature'].isna()==False) & (dt_tw['air_temperature'].isna()==True))|\
     ((dt_tw['dew_temperature'].isna()==True) & (dt_tw['air_temperature'].isna()==False))]
ind=mis_tw.index
for i in ind :
    if pd.isnull(dt_tw.at[i,'dew_temperature']):
        dt_tw.at[i,'dew_temperature']=model_tr.predict(np.array([1,dt_tw.at[i,'air_temperature']]).\
                                                       reshape(1,-1))
    else:
        dt_tw.iloc.at[i,'air_temperature']=(dt_tw.at[i,'dew_temperature']-intertr)/coeftr[1]


In [ ]:
#Impute test missing values .
mis_tsw=dt_tsw[((dt_tsw['dew_temperature'].isna()==False) & (dt_tsw['air_temperature'].isna()==True))|\
     ((dt_tsw['dew_temperature'].isna()==True) & (dt_tsw['air_temperature'].isna()==False))]
indts=mis_tsw.index
for i in indts :
    if pd.isnull(dt_tsw.at[i,'dew_temperature']):
        dt_tsw.at [i,'dew_temperature']=model_tr.predict(np.array([1,dt_tsw.at[i,'air_temperature']]).\
                                                       reshape(1,-1))
    else:
        dt_tsw.at [i,'air_temperature']=(dt_tsw.at[i,'dew_temperature']-intertr)/coeftr[1]

Thank to high colinearity between the variables air_temperature and dew_temperature. We will save only the variable air_temperature , which known to have more influence on the target variable.

In [ ]:
del(dt_tw['dew_temperature'])
del(dt_tsw['dew_temperature'])

For the rest of missing values , we will try to find the near values of the missing one. Indeed, the nearst wich will be defined in function of site_id and the other time parameters. For this reason we will use the KnearstNeighbors algorhitm to imput theses missing values .

In [ ]:
# Heureunder , we will define new variables which can allow to decorticate times parameters.
dt_tw['month']=pd.to_datetime(dt_tw['timestamp']).dt.month
dt_tw['day']=pd.to_datetime(dt_tw['timestamp']).dt.day
dt_tw['hour']=pd.to_datetime(dt_tw['timestamp']).dt.hour
dt_tw['day_week']=pd.to_datetime(dt_tw['timestamp']).dt.dayofweek

In [ ]:
#We will do same thing for test weather .
dt_tsw['month']=pd.to_datetime(dt_tsw['timestamp']).dt.month
dt_tsw['day']=pd.to_datetime(dt_tsw['timestamp']).dt.day
dt_tsw['hour']=pd.to_datetime(dt_tsw['timestamp']).dt.hour
dt_tsw['day_week']=pd.to_datetime(dt_tsw['timestamp']).dt.dayofweek

In [ ]:
def training_values(vr):
    """This function allow to determinate the training data , which could be used to train further 
    alghorithm in order to predict the target value of the variable introduced as parameter.
    
    Args:
    
    vr(string): The name of the target variable, that we want to predict .
    
    Returns:
    
    X,Y : The train data extracted from dt_tw and dt_tsw , to serve in training an machine learning alghorithm.
    """
    id_tr=dt_tw[dt_tw[vr].notna()==True].index
    id_ts=dt_tsw[dt_tsw[vr].notna()==True].index
    X=dt_tw.iloc[id_tr][['site_id','month','day','hour']]
    X=X.append(dt_tsw.iloc[id_ts][['site_id','month','day','hour']])
    Y=dt_tw[vr].iloc[id_tr]
    Y=Y.append(dt_tsw[vr].iloc[id_ts])
    return X,Y

In [ ]:
def imputer(var,algo,df1,df2):
    """ This function , allow to impute the missing values of the variable namely var, which introduced as
        parameter in the dataframes df1 and df2 .This on base of the trained machine learning algo, which 
        had introduced as parameter.
        
        Args:
        
        vr(string): the name of the variable , which we will impute his missing values .
        algo : the trained machine learning , which will predict the variable missing values .
        df1,df2(DataFrame): the dataframe, that we will impute on it the missing values .
        
        """
    ii_tr=df1[df1[var].isna()==True].index
    ii_ts=df2[df2[var].isna()==True].index
    for i in ii_tr :
        df1.at[i,var]=algo.predict(df1.iloc[i][['site_id','month','day','hour']].to_numpy().reshape(1,-1))
    for j in ii_ts :
        df2.at[j,var]=algo.predict(df2.iloc[j][['site_id','month','day','hour']].to_numpy().reshape(1,-1))

#### air_temperature missing values

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
X,Y=training_values('air_temperature')

In [ ]:
imputer_temperature=KNeighborsRegressor()
imputer_temperature.fit(X,Y)

In [ ]:
#Impute air_temperature missing values .
imputer('air_temperature',imputer_temperature,dt_tw,dt_tsw)


#### sea_level_pressure missing values 

In [ ]:
X1,Y1=training_values('sea_level_pressure')

In [ ]:
imputer_pressure=KNeighborsRegressor()
imputer_pressure.fit(X1,Y1)

In [ ]:
#Impute sea_level_pressure missing values .
imputer('sea_level_pressure',imputer_pressure,dt_tw,dt_tsw)

#### Wind direction missing values 

In [ ]:
X2,Y2=training_values('wind_direction')

In [ ]:
imputer_wind_direction=KNeighborsRegressor()
imputer_wind_direction.fit(X2,Y2)

In [ ]:
#Impute wind_direction missing values .
imputer('wind_direction',imputer_wind_direction,dt_tw,dt_tsw)

#### wind speed missing values 

In [ ]:
X3,Y3=training_values('wind_speed')

In [ ]:
imputer_wind_speed=KNeighborsRegressor()
imputer_wind_speed.fit(X3,Y3)

In [ ]:
#Impute wind_speed missing values .
imputer('wind_direction',imputer_wind_speed,dt_tw,dt_tsw)

# Outliers:

In [ ]:
medianprops={'color':"black"}
meanprops={'marker':'o','markeredgecolor':'black','markerfacecolor':'firebrick'}
plt.boxplot(dt_tr['meter_reading'],labels=['meter_reading'],vert=False ,medianprops=medianprops,\
           meanprops=meanprops,showmeans=True,showfliers=False,patch_artist=True)

We notice from the boxplot above , that there is meter_reading with zero values . Theses values , seem to be outliers values . So we will drop all rows wich contains meter_reading with zero value .

In [ ]:
dt_tr=dt_tr[dt_tr['meter_reading']!=0]

# Prepare datas :

<font style=Italy> <font color=green>Hereunder , we will make jointure between  dt_tr ,dt_bd, dt_tw , in order to bring together all training datas in the same dataframe table . Besides , we do the same work for test datas .</font>

In [ ]:
# Here we will do an left external Juncture between dt_tr,dt_bd, 
# which traduct an association of the buiding consumption datas 
# with the datas of building carateristic .
tr1=pd.merge(dt_tr,dt_bd,how='left',on='building_id')

In [ ]:
# Now , we will do an left external juncture between the last 
# obtained datafram tr1 , with dt_tw, which will allow to 
# associate the building consumption with the weather conditions.
tr=pd.merge(tr1,dt_tw,on=['site_id','timestamp'],how='left')

In [ ]:
tr[tr['site_id']==0]['meter_reading']=[0.2931* c for c in (tr[tr['site_id']==0]['meter_reading'])]

In [ ]:
# Now we will the same work for the test datas.
ts1=pd.merge(dt_ts,dt_bd,on='building_id',how='left')
ts=pd.merge(ts1,dt_tsw,on=['site_id','timestamp'],how='left')

In [ ]:
'''Function to reduce the DF size'''
# source: https://www.kaggle.com/kernels/scriptcontent/3684066/download

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
dt_tr=reduce_mem_usage(dt_tr)
dt_bd=reduce_mem_usage(dt_bd)
dt_tw=reduce_mem_usage(dt_tw)
dt_ts=reduce_mem_usage(dt_ts)
dt_tsw=reduce_mem_usage(dt_tsw)

In [ ]:
# We will delete all useles dataframes to reduce memory :
del(dt_tr)
del(dt_bd)
del(dt_tw)
del(dt_ts)
del(dt_tsw)

In [ ]:
ts1=reduce_mem_usage(ts1)
tr1=reduce_mem_usage(tr1)

#### Missing values

In [ ]:
from datetime import datetime
# time missing values
mtr=tr[(tr['month'].isna()==True)|(tr['day'].isna()==True)|(tr['hour'].isna()==True)|\
      (tr['day_week'].isna()==True)].index # Note the same index for all variables missing values 
mts=ts[(ts['month'].isna()==True)|(ts['day'].isna()==True)|(ts['hour'].isna()==True)|\
      (ts['day_week'].isna()==True)].index
for i in mtr :
    tr.at[i,'month']=datetime.strptime(tr.at[i,'timestamp'],'%Y-%m-%d %H:%M:%S').month
    tr.at[i,'day']=datetime.strptime(tr.at[i,'timestamp'],'%Y-%m-%d %H:%M:%S').day
    tr.at[i,'hour']=datetime.strptime(tr.at[i,'timestamp'],'%Y-%m-%d %H:%M:%S').hour
    tr.at[i,'day_week']=datetime.strptime(tr.at[i,'timestamp'],'%Y-%m-%d %H:%M:%S').weekday()
for i in mts :
    ts.at[i,'month']=datetime.strptime(ts.at[i,'timestamp'],'%Y-%m-%d %H:%M:%S').month
    ts.at[i,'day']=datetime.strptime(ts.at[i,'timestamp'],'%Y-%m-%d %H:%M:%S').day
    ts.at[i,'hour']=datetime.strptime(ts.at[i,'timestamp'],'%Y-%m-%d %H:%M:%S').hour
    ts.at[i,'day_week']=datetime.strptime(ts.at[i,'timestamp'],'%Y-%m-%d %H:%M:%S').weekday()

We will use the above missing values predictor to impute the new missing values.

In [ ]:
# impute air_temperature missing values .
trt=tr[tr['air_temperature'].isna()==True].index
tst=ts[ts['air_temperature'].isna()==True].index
for i in trt :
    tr.at[i,'air_temperature']=imputer_temperature.predict(tr.iloc[i][['site_id','month','day','hour']].to_numpy().reshape(1,-1))    
for i in tst :
    ts.at[i,'air_temperature']=imputer_temperature.predict(ts.iloc[i][['site_id','month','day','hour']].to_numpy().reshape(1,-1))
    

In [ ]:
# impute sea_level_pressure missing values .
prt=tr[tr['sea_level_pressure'].isna()==True].index
pst=ts[ts['sea_level_pressure'].isna()==True].index
for i in prt :
    tr.at[i,'sea_level_pressure']=imputer_pressure.predict(tr.iloc[i][['site_id','month','day','hour']].to_numpy().reshape(1,-1))
for i in pst :
    ts.at[i,'sea_level_pressure']=imputer_pressure.predict(ts.iloc[i][['site_id','month','day','hour']].to_numpy().reshape(1,-1))
    

In [ ]:
# impute wind_direction missing values.
drt=tr[tr['wind_direction'].isna()==True].index
dst=ts[ts['wind_direction'].isna()==True].index
for i in drt :
    tr.at[i,'wind_direction']=imputer_wind_direction.predict(tr.iloc[i][['site_id','month','day','hour']].to_numpy().reshape(1,-1))
for i in dst :
    ts.at[i,'wind_direction']=imputer_wind_direction.predict(ts.iloc[i][['site_id','month','day','hour']].to_numpy().reshape(1,-1))

In [ ]:
# impute wind_speed missing values .
srt=tr[tr['wind_speed'].isna()==True].index
sts=ts[ts['wind_speed'].isna()==True].index
for i in srt :
    tr.at[i,'wind_speed']=imputer_wind_speed.predict(tr.iloc[i][['site_id','month','day','hour']].to_numpy().reshape(1,-1))
for i in sts :
    ts.at[i,'wind_speed']=imputer_wind_speed.predict(ts.iloc[i][['site_id','month','day','hour']].to_numpy().reshape(1,-1))

# Features engineering: 

In [ ]:
# Let 's visualize the meter_reding in function of month
freq=tr['month'].value_counts()
mois=list(freq.index)
mois_consumption=np.array([tr[tr['month']==c]['meter_reading'].sum()/freq.loc[c] for c in mois])
ind=np.argsort(mois_consumption)
month=[]
month_consumption=[]
for i in ind :
    month+=[str(mois[i])]
    month_consumption+=[mois_consumption[i]]
plt.bar(month,month_consumption)
plt.xlabel('Month',fontsize=12,color='red')
plt.ylabel('Mean energy consumption',fontsize=12,color='red')
plt.title('Mean energy consumption per month',color='orange',fontsize=14)
plt.show()

In [ ]:
encodage_month={7:1,8:2,10:3,12:4,9:5,11:6,1:7,2:8,6:9,3:10,5:11,4:12}
tr['month']=[encodage_month[c] for c in tr['month']]

In [ ]:
# Let 's visualize the meter_reding in function of day of week
freq=tr['day_week'].value_counts()
jours=list(freq.index)
jours_consumption=np.array([tr[tr['day_week']==c]['meter_reading'].sum()/freq.loc[c] for c in jours])
ind=np.argsort(jours_consumption)
days=[]
days_consumption=[]
for i in ind :
    days+=[str(jours[i])]
    days_consumption+=[jours_consumption[i]]
plt.bar(days,days_consumption)
plt.xlabel('Day of week ',fontsize=12,color='red')
plt.ylabel('Mean energy consumption',fontsize=12,color='red')
plt.title('Mean energy consumption per day of week',color='orange',fontsize=14)
plt.show()

In [ ]:
tr['month'].unique()

In [ ]:
encodage_days_week={6:1,5:2,0:3,4:4,3:5,1:6,2:7}
tr['day_week']=[encodage_days_week[c] for c in tr['day_week']]

In [ ]:
# Let 's visualize the meter_reding in function of day hours
fig=plt.figure(figsize=(10,8))
freq=tr['hour'].value_counts()
heures=list(freq.index)
heures_consumption=np.array([tr[tr['hour']==c]['meter_reading'].sum()/freq.loc[c] for c in heures])
ind=np.argsort(heures_consumption)
hours=[]
hours_consumption=[]
for i in ind :
    hours+=[str(heures[i])]
    hours_consumption+=[heures_consumption[i]]
plt.barh(hours,hours_consumption)
plt.ylabel('Hours',fontsize=12,color='red')
plt.xlabel('Mean energy consumption',fontsize=12,color='red')
plt.title('Mean energy consumption per hour',color='orange',fontsize=14)
plt.show()

In [ ]:
encodage_hours={6:1,5:2,3:3,4:4,7:5,8:6,2:7,1:8,0:9,9:10,10:11,23:12,11:13,12:14,22:15,13:16,14:17,17:18,21:19,\
               18:20,20:21,19:22,16:23,15:24}
tr['hour']=[encodage_hours[c] for c in tr['hour']]

In [ ]:
# Let 's visualize the meter_reding in function of day of month 
fig=plt.figure(figsize=(10,12))
freq=tr['day'].value_counts()
jmois=list(freq.index)
jmois_consumption=np.array([tr[tr['day']==c]['meter_reading'].sum()/freq.loc[c] for c in jmois])
ind=np.argsort(jmois_consumption)
dmonth=[]
dmonth_consumption=[]
for i in ind :
    dmonth+=[str(jmois[i])]
    dmonth_consumption+=[jmois_consumption[i]]
plt.barh(dmonth,dmonth_consumption)
plt.ylabel('Day of month',fontsize=12,color='red')
plt.xlabel('Mean energy consumption',fontsize=12,color='red')
plt.title('Mean energy consumption per day of month',color='orange',fontsize=14)
plt.show()

In [ ]:
encodage_day_month={17:1,18:2,4:3,2:4,19:5,20:6,5:7,21:8,1:9,22:10,30:11,23:12,3:13,26:14,25:15,24:16,31:17,6:18,\
                   7:19,29:20,28:21,27:22,16:23,11:24,15:25,12:26,14:27,8:28,13:29,10:30,9:31}
tr['day']=[encodage_day_month[c] for c in tr['day']]

In [ ]:
del(tr['timestamp']) # useless variable
tr=reduce_mem_usage(tr) # reduce memory

In [ ]:
ts['month']=[encodage_month[c] for c in ts['month']]
ts['day']=[encodage_day_month[c] for c in ts['day']]
ts['hour']=[encodage_hours[c] for c in ts['hour']]
ts['day_week']=[encodage_days_week[c] for c in ts['day_week']]

In [ ]:
del(ts['timestamp']) # useless variable
ts=reduce_mem_usage(ts) # reduce memory

In [ ]:
# Hereunder , we will visualize the mean hour consumption energy per primary use.
fig=plt.figure(figsize=(15,10))
tr_user=list(tr['primary_use'].unique())
frequency_use=tr['primary_use'].value_counts()
user_consumption=np.array([((((tr[tr['primary_use']==c])['meter_reading']).sum())/frequency_use.loc[c])\
                  for c in tr_user ])
ind=np.argsort(user_consumption)
user=[]
consumption=[]
for i in ind :
    user +=[tr_user[i]]
    consumption+=[user_consumption[i]]
plt.barh(user,consumption,color='red')
plt.ylabel('primary_use',fontsize=18)
plt.xlabel('meter_reading',fontsize=18)
plt.title('consumption energy per primary_use for training datas',color='blue',fontsize=18)


The chart above , could help us to give a meaningful encoding of the primary_use variable , properly  to the mean consumption of each type of primary use building . So we will use the following encoding of this variable, wich allow to give weight in concordance with each type primary consumption.  

Primary_use | Weights
----------- |--------
Religious worship | 1
Warehouse/storage | 2
Other| 3
Technology/science | 4
Retail            | 5
Parking           | 6
Manufacturing industrial | 7
Public services | 8
Lodging/residential      | 9
Food sales and services  | 10
Utility | 11
Entertainement /public assembly | 12
Office  | 13
Healthcare | 14
Services | 15
Education | 16

In [ ]:
encodage={'Religious worship':1,'Warehouse/storage':2,'Technology/science':4,'Other':3,'Retail':5,'Parking':6,\
         'Lodging/residential':9,'Manufacturing/industrial':7,'Public services':8,'Food sales and service':10,\
         'Entertainment/public assembly':12,'Utility':11,'Office':13,'Healthcare':14,'Services':15,\
         'Education':16}
tr['primary_use']=[encodage[c] for c in tr['primary_use']]
ts['primary_use']=[encodage[c] for c in ts['primary_use']]

In [ ]:
# Hereunder , we will visualize the mean hour consumption energy per type of consumption .
fig=plt.figure(figsize=(15,10))
tr_type=[0,1,2,3]
type_cor={0:'electricity',1:'chilledwater',2:'steam',3:'hotwater'}
type_freq_use=tr['meter'].value_counts()
type_consumption=np.array([(tr[tr['meter']==c]['meter_reading'].sum())/(type_freq_use[c]) for c in tr_type])
idn=np.argsort(type_consumption)
cs=[]
typ=[]
for i in idn :
    cs+=[type_consumption[i]]
    typ+=[type_cor[tr_type[i]]]
plt.bar(typ,cs,color='orange')
plt.xlabel('Energy type',fontsize=18)
plt.ylabel('Energy consumption',fontsize=18)
plt.title('Energy consumption per energy type',fontsize=18,color='blue')
plt.show()

The chart above , will help us to reencode the variable titled meter , in concordance with the mean energy consumption of each energy type.  

Type  | Weight 
----- | -------
electricity | 1
hotwater | 2
chilledwater | 3
steam | 4

In [ ]:
meter_encodage={0:1,3:2,1:3,2:4}
tr['meter']=[meter_encodage[c] for c in tr['meter']]
ts['meter']=[meter_encodage[c] for c in ts['meter']]

# Analyse multivariate :

### Quantitative variables :

In [ ]:
# We analyse the correlation between the quantitative variables .
cor=(tr[['meter_reading','square_feet','air_temperature','sea_level_pressure',\
      'wind_direction','wind_speed']]).corr()
cor

# Baseline performance

## Linear regression

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV


We will test a Ridge linear regression , so will do the following steps :
* We will use Gridsearchcv to tunning the best hyperparameter.
* split the data into train and test datas .
* perform the baseline performance of the ridge regression 

In [ ]:
# Prepare the datas .
train =tr.drop(columns=['meter_reading','building_id','site_id'])
test=ts.drop(columns=['building_id','site_id'])
target=tr['meter_reading']

In [ ]:
from sklearn.model_selection import train_test_split
Xtr,Xts,Ytr,Yts=train_test_split(train,target,test_size=0.2)

In [ ]:
# tuning the best hyperparametrs
linear=Ridge()
parametrs={'alpha':np.logspace(-3,3,10)}
reg_lin=GridSearchCV(linear,parametrs,cv=5)
reg_lin.fit(Xtr,Ytr)

In [ ]:
sc1=reg_lin.score(Xts,Yts)
print('The baseline score is : {}'.format(sc1))

In [ ]:
test=test.drop(columns='row_id')
samples=pd.DataFrame({'row_id':ts['row_id']})
samples['meter_reading']=reg_lin.predict(test)

In [ ]:
samples.to_csv('ridge.csv',index=False)

In [ ]:
tr.to_csv('tr.csv',index=False)

In [ ]:
ts.to_csv('ts.csv',index=False)